# Configure Sugar


In [ ]:
#| default_exp config

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import os
from dataclasses import dataclass, make_dataclass
from sugar.helpers import normalize_address
from typing import List

In [ ]:
#| export

base_default_settings = {
  "price_batch_size": int(os.getenv("SUGAR_PRICE_BATCH_SIZE","40")),
  "price_threshold_filter": int(os.getenv("SUGAR_PRICE_THRESHOLD_FILTER","10")),
  "pagination_limit": int(os.getenv("SUGAR_PAGINATION_LIMIT","2000")),
  "pool_page_size": int(os.getenv("SUGAR_POOL_PAGE_SIZE","500")),
  "native_token_symbol": "ETH",
  "native_token_decimals": 18
}


@dataclass
class ChainSettings:
    # chain IDs come from: https://chainlist.org/
    chain_id: str
    chain_name: str
    wrapped_native_token_addr: str
    rpc_uri: str
    sugar_contract_addr: str
    # slipstream operates on concentrated liquidity (CL) pools
    slipstream_contract_addr: str
    # Non-Fungible Position Manager for CL pools
    nfpm_contract_addr: str
    price_oracle_contract_addr: str
    router_contract_addr: str
    token_addr: str
    stable_token_addr: str
    connector_tokens_addrs: List[str]
    price_batch_size: int
    price_threshold_filter: int
    pagination_limit: int
    pool_page_size: int
    native_token_symbol: str
    native_token_decimals: int

    def __str__(self):
        # go over all attributes of self
        lines = ["🍭 Chain settings:","----------------"]
        attributes = [attr for attr in dir(self) if not attr.startswith('_')]
        for attr in attributes:
            # Skip methods
            if callable(getattr(self, attr)): continue
            value = getattr(self, attr)
            if isinstance(value, list):
                value = "\n" + "\n".join([f"{' ' * 4 }- {v}" for v in value])
            lines.append(f"{attr}: {value}")
        return "\n".join(lines)
    
    def __repr__(self): return str(self)

def validate_settings(settings: ChainSettings) -> ChainSettings:
    return settings


def make_settings(chain_id: str, chain_name: str, **kwargs) -> ChainSettings:
    d = { **base_default_settings, **kwargs, **{ "chain_id": chain_id, "chain_name": chain_name } }
    for k,v in d.items():
        if k in ["chain_id", "chain_name"]: continue
        # look for env vars
        # e.g stable_token_addr for chain_id="10" should be SUGAR_STABLE_TOKEN_ADDR_10
        d[k] = os.getenv(f"SUGAR_{k.upper()}_{chain_id}", v)
        # anything that ends in "_addr", should be normalized
        if k.endswith("_addr"): d[k] = normalize_address(d[k])
        # anything that ends in "_addrs", should be a list of normalized addresses
        if k.endswith("_addrs"): d[k] = list(map(lambda a: normalize_address(a), d[k].split(",")))
    return validate_settings(make_dataclass(ChainSettings.__name__, ((k, type(v)) for k, v in d.items()))(**d)) 

## OP Chain settings

In [ ]:
#| export

def make_op_chain_settings(**kwargs) -> ChainSettings:
    d = {
        "rpc_uri": "https://optimism-mainnet.wallet.coinbase.com",
        "wrapped_native_token_addr": "0x4200000000000000000000000000000000000006",
        "sugar_contract_addr": "0x3B919747B46B13CFfd9f16629cFf951C0b7ea1e2",
        "slipstream_contract_addr": "0xD45624bf2CB9f65ecbdF3067d21992b099b56202",
        "nfpm_contract_addr": "0x416b433906b1B72FA758e166e239c43d68dC6F29",
        "price_oracle_contract_addr": "0x59114D308C6DE4A84F5F8cD80485a5481047b99f",
        "router_contract_addr": "0xa062aE8A9c5e11aaA026fc2670B0D65cCc8B2858",
        "token_addr": "0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db",
        "stable_token_addr": "0x7F5c764cBc14f9669B88837ca1490cCa17c31607",
        "connector_tokens_addrs": "0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db,0x4200000000000000000000000000000000000042,0x4200000000000000000000000000000000000006,0x9bcef72be871e61ed4fbbc7630889bee758eb81d,0x2e3d870790dc77a83dd1d18184acc7439a53f475,0x8c6f28f2f1a3c87f0f938b96d27520d9751ec8d9,0x1f32b1c2345538c0c6f582fcb022739c4a194ebb,0xbfd291da8a403daaf7e5e9dc1ec0aceacd4848b9,0xc3864f98f2a61a7caeb95b039d031b4e2f55e0e9,0x9485aca5bbbe1667ad97c7fe7c4531a624c8b1ed,0xDA10009cBd5D07dd0CeCc66161FC93D7c9000da1,0x73cb180bf0521828d8849bc8cf2b920918e23032,0x6806411765af15bddd26f8f544a34cc40cb9838b,0x6c2f7b6110a37b3b0fbdd811876be368df02e8b0,0xc5b001dc33727f8f26880b184090d3e252470d45,0x6c84a8f1c29108f47a79964b5fe888d4f4d0de40,0xc40f949f8a4e094d1b49a23ea9241d289b7b2819,0x94b008aa00579c1307b0ef2c499ad98a8ce58e58,0x0b2c639c533813f4aa9d7837caf62653d097ff85" 
    } 
    return make_settings("10", "OP", **{ **d, **kwargs })

## BASE Chain settings

In [ ]:
#| export

def make_base_chain_settings(**kwargs) -> ChainSettings:
    d = {
        "rpc_uri": "https://mainnet.base.org",
        "wrapped_native_token_addr": "0x4200000000000000000000000000000000000006",
        "sugar_contract_addr": "0x92294D631E995f1dd9CeE4097426e6a71aB87Bcf",
        "slipstream_contract_addr": "0x0AD09A66af0154a84e86F761313d02d0abB6edd5",
        "nfpm_contract_addr": "0x827922686190790b37229fd06084350E74485b72",
        "price_oracle_contract_addr": "0x3B06c787711ecb5624cE65AC8F26cde10831eb0C",
        "router_contract_addr": "0xcF77a3Ba9A5CA399B7c97c74d54e5b1Beb874E43",
        "token_addr": "0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db",
        "stable_token_addr": "0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913",
        "connector_tokens_addrs": "0x833589fcd6edb6e08f4c7c32d4f71b54bda02913,0x940181a94A35A4569E4529A3CDfB74e38FD98631,0x50c5725949A6F0c72E6C4a641F24049A917DB0Cb,0x4621b7a9c75199271f773ebd9a499dbd165c3191,0x4200000000000000000000000000000000000006,0xb79dd08ea68a908a97220c76d19a6aa9cbde4376,0xf7a0dd3317535ec4f4d29adf9d620b3d8d5d5069,0xcfa3ef56d303ae4faaba0592388f19d7c3399fb4,0xcb327b99ff831bf8223cced12b1338ff3aa322ff,0x2ae3f1ec7f1f5012cfeab0185bfc7aa3cf0dec22,0xc1cba3fcea344f92d9239c08c0568f6f2f0ee452,0x60a3e35cc302bfa44cb288bc5a4f316fdb1adb42,0xd9aaec86b65d86f6a7b5b1b0c42ffa531710b6ca,0xcbB7C0000aB88B473b1f5aFd9ef808440eed33Bf"
    }
    return make_settings("8453", "Base", **{ **d, **kwargs })

In [ ]:
#| hide

import nbdev; nbdev.nbdev_export()